In [30]:
import pandas as pd
import numpy as np
import datetime

In [2]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from src.utils import eda

In [6]:
df_incidentes = pd.read_csv('../data/incidentes-viales-c5.csv')

In [7]:
df_incidentes.sample()

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
518226,C4/140219/01029,19/02/2014,11:22:27,Miércoles,(A) La unidad de atención a emergencias fue de...,19/02/2014,2014,Febrero,11:40:24,IZTAPALAPA,accidente-choque sin lesionados,19.37887,-99.07733,EMERGENCIA,LLAMADA DEL 066,IZTAPALAPA,"19.37887,-99.07733",2


In [10]:
len(df_incidentes)

1383138

Se tienen 1,383,138 registros de incidentes viales en la ciudad de México.

In [8]:
df_incidentes['folio'].nunique()

1383138

El folio efectivamente identifica a cada renglón con un valor único.

In [11]:
len(df_incidentes.columns)

18

Se tienen 18 columnas o variables, las cuales son:

In [12]:
list(df_incidentes.columns)

['folio',
 'fecha_creacion',
 'hora_creacion',
 'dia_semana',
 'codigo_cierre',
 'fecha_cierre',
 'año_cierre',
 'mes_cierre',
 'hora_cierre',
 'delegacion_inicio',
 'incidente_c4',
 'latitud',
 'longitud',
 'clas_con_f_alarma',
 'tipo_entrada',
 'delegacion_cierre',
 'geopoint',
 'mes']

Los tipos de dato que tiene cada variable inicialmente son:

In [13]:
df_incidentes.dtypes

folio                 object
fecha_creacion        object
hora_creacion         object
dia_semana            object
codigo_cierre         object
fecha_cierre          object
año_cierre             int64
mes_cierre            object
hora_cierre           object
delegacion_inicio     object
incidente_c4          object
latitud              float64
longitud             float64
clas_con_f_alarma     object
tipo_entrada          object
delegacion_cierre     object
geopoint              object
mes                    int64
dtype: object

## Perfilamiento general

In [14]:
eda.genera_profiling_general(df_incidentes)

El dataframe tiene 18 columnas.
Hay 5 columnas que tienen NA's.


,Estadisticas,Resultado
0,Total de variables,18
1,Conteo de observaciones,1383138
2,Total de celdas,24896484
3,Cantidad de variables numericas,4
4,Cantidad de variables de fecha,0
5,Cantidad de variables categóricas,0
6,Cantidad de variables de texto,14
7,Valores faltantes,1622
8,Porcentaje de valores faltantes,0.0%
9,Renglones duplicados,0


En la base de datos, tenemos alrededor de 1,622 elementos faltantes, distribuidos de la siguiente forma:

In [15]:
eda.cuenta_nulos_por_columnas(df_incidentes)

El dataframe tiene 18 columnas.
Hay 5 columnas que tienen NA's.


,Missing Values,% del Total
geopoint,446,0.0
latitud,443,0.0
longitud,435,0.0
delegacion_inicio,158,0.0
delegacion_cierre,140,0.0


En cuanto a los tipos de datos, están de la siguiente forma:

In [16]:
df_incidentes.dtypes

folio                 object
fecha_creacion        object
hora_creacion         object
dia_semana            object
codigo_cierre         object
fecha_cierre          object
año_cierre             int64
mes_cierre            object
hora_cierre           object
delegacion_inicio     object
incidente_c4          object
latitud              float64
longitud             float64
clas_con_f_alarma     object
tipo_entrada          object
delegacion_cierre     object
geopoint              object
mes                    int64
dtype: object

folio                 object --> Object  
fecha_creacion        object --> Date  
hora_creacion         object --> Date  
dia_semana            object --> Category  
codigo_cierre         object --> Category  
fecha_cierre          object --> Date  
año_cierre             int64 --> Category  
mes_cierre            object --> Category  
hora_cierre           object --> Date  
delegacion_inicio     object --> Category  
incidente_c4          object --> Category  
latitud              float64 --> float  
longitud             float64 --> float  
clas_con_f_alarma     object --> Category  
tipo_entrada          object --> Category  
delegacion_cierre     object --> Category  
geopoint              object --> Object  
mes                    int64 --> Category  

Primero, arreglamos los strings en las fechas:

In [20]:
# fix Dates
for ano in range(2013,2021):
    string_correccion_ano = str(ano)
    string_correccion_ano=string_correccion_ano[-2:]
    print(string_correccion_ano)
    
    for mes in range (1,13):
        if(mes<10):
            string_correccion_mes = '0'+str(mes)
        else:
            string_correccion_mes = str(mes)
            
        for dia in range(1, 32):
            elemento = dia

            if(elemento<10):
                string_correccion = '0'+str(dia)+'/'+string_correccion_mes+'/'+string_correccion_ano
                string_correcto = '0'+str(dia)+'/'+string_correccion_mes+'/20'+string_correccion_ano
            else:
                string_correccion = str(dia)+'/'+string_correccion_mes+'/'+string_correccion_ano
                string_correcto = str(dia)+'/'+string_correccion_mes+'/20'+string_correccion_ano
            #print(string_correccion)
            df_incidentes.loc[df_incidentes["fecha_creacion"] == string_correccion, "fecha_creacion"] = string_correcto
            df_incidentes.loc[df_incidentes["fecha_cierre"] == string_correccion, "fecha_cierre"] = string_correcto

13
14
15
16
17
18
19
20


In [21]:
# Convertir columna en datetime 
df_incidentes['fecha_creacion']=pd.to_datetime(df_incidentes['fecha_creacion'], format='%d/%m/%Y')
df_incidentes['fecha_cierre']=pd.to_datetime(df_incidentes['fecha_cierre'], format='%d/%m/%Y')

Ahora, corregimos las horas de creación y de cierre.

In [22]:
df_incidentes['hora_creacion'].unique()

array(['22:35:04', '22:50:49', '09:40:11', ..., '0.986863426',
       '0.996238426', '04:33:05'], dtype=object)

In [23]:
df_incidentes['hora_creacion'].min()

'0.000138889'

In [24]:
df_hrs_raras = df_incidentes.loc[~df_incidentes["hora_creacion"].str.contains(':', case=False, na=None)]

In [25]:
df_hrs_raras.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
831339,C5/200217/09463,2020-02-17,0.985266204,Lunes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.147476852,MIGUEL HIDALGO,accidente-choque sin lesionados,19.45393,-99.19447,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.45393002,-99.19446984",2
831341,C5/200218/03696,2020-02-18,0.53494213,Martes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.592511574,AZCAPOTZALCO,lesionado-atropellado,19.49486,-99.19591,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,"19.49486004,-99.19590984",2


In [26]:
len(df_hrs_raras)

1544

In [27]:
hrs_raras = df_hrs_raras['hora_creacion']

In [28]:
hrs_raras

831339     0.985266204
831341      0.53494213
831343     0.366539352
831345      0.51306713
831347     0.541273148
              ...     
1382352    0.921678241
1382353    0.967268519
1382354    0.976018519
1382355    0.986863426
1382356    0.996238426
Name: hora_creacion, Length: 1544, dtype: object

In [31]:
print(datetime.timedelta(days=0.985266204))

23:38:47.000026


In [32]:
print(datetime.timedelta(days=float(list(hrs_raras)[0])))

23:38:47.000026


In [33]:
datetime.timedelta(days=float(list(hrs_raras)[0]))

datetime.timedelta(seconds=85127, microseconds=26)

In [34]:
hrs_raras = hrs_raras.astype('float')
hours = [datetime.timedelta(days=float(num)) for num in hrs_raras]

In [35]:
data_hr_corregida = {'folio':df_hrs_raras['folio'],'hora_creacion':df_hrs_raras['hora_creacion'],'hora_creacion_corregida':hours}

In [36]:
dataframe_hrcorregida = pd.DataFrame(data=data_hr_corregida)

In [37]:
dataframe_hrcorregida = df_hrs_raras.copy()
dataframe_hrcorregida['hora_creacion_corregida'] = hours

In [38]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida
831339,C5/200217/09463,2020-02-17,0.985266204,Lunes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.147476852,MIGUEL HIDALGO,accidente-choque sin lesionados,19.45393,-99.19447,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.45393002,-99.19446984",2,0 days 23:38:47.000026
831341,C5/200218/03696,2020-02-18,0.53494213,Martes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.592511574,AZCAPOTZALCO,lesionado-atropellado,19.49486,-99.19591,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,"19.49486004,-99.19590984",2,0 days 12:50:19.000032


In [39]:
# Eliminamos microsegundos
dataframe_hrcorregida['hora_creacion_corregida']= dataframe_hrcorregida['hora_creacion_corregida'].dt.floor('s')

In [40]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida
831339,C5/200217/09463,2020-02-17,0.985266204,Lunes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.147476852,MIGUEL HIDALGO,accidente-choque sin lesionados,19.45393,-99.19447,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.45393002,-99.19446984",2,0 days 23:38:47
831341,C5/200218/03696,2020-02-18,0.53494213,Martes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.592511574,AZCAPOTZALCO,lesionado-atropellado,19.49486,-99.19591,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,"19.49486004,-99.19590984",2,0 days 12:50:19


In [41]:
dataframe_hrcorregida.dtypes

folio                               object
fecha_creacion              datetime64[ns]
hora_creacion                       object
dia_semana                          object
codigo_cierre                       object
fecha_cierre                datetime64[ns]
año_cierre                           int64
mes_cierre                          object
hora_cierre                         object
delegacion_inicio                   object
incidente_c4                        object
latitud                            float64
longitud                           float64
clas_con_f_alarma                   object
tipo_entrada                        object
delegacion_cierre                   object
geopoint                            object
mes                                  int64
hora_creacion_corregida    timedelta64[ns]
dtype: object

In [42]:
#La pasamos a string para eliminar el texto de 0 días
dataframe_hrcorregida['hora_creacion_corregida'] = dataframe_hrcorregida['hora_creacion_corregida'].astype(str)

In [43]:
#Eliminamos el string de 0 dias
dataframe_hrcorregida['hora_creacion_corregida'] = dataframe_hrcorregida['hora_creacion_corregida'].str.replace('0 days ', '')

In [44]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida
831339,C5/200217/09463,2020-02-17,0.985266204,Lunes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.147476852,MIGUEL HIDALGO,accidente-choque sin lesionados,19.45393,-99.19447,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.45393002,-99.19446984",2,23:38:47
831341,C5/200218/03696,2020-02-18,0.53494213,Martes,(A) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.592511574,AZCAPOTZALCO,lesionado-atropellado,19.49486,-99.19591,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,"19.49486004,-99.19590984",2,12:50:19


In [45]:
dataframe_hrcorregida['hora_creacion'] = dataframe_hrcorregida['hora_creacion_corregida']

In [46]:
df_incidentes = df_incidentes.append(dataframe_hrcorregida)

In [47]:
df_incidentes.drop_duplicates(keep='last',subset=['folio'],inplace = True)

In [48]:
df_incidentes.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida
0,GA/160123/05714,2016-01-23,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,2016-01-24,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,"19.422113,-99.084643",1,NaN
1,AO/160123/05826,2016-01-23,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,2016-01-24,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,"19.35832,-99.297641",1,NaN


In [49]:
# Verificamos:
df_incidentes['hora_creacion'].unique()

array(['22:35:04', '22:50:49', '09:40:11', ..., '03:08:57', '02:05:12',
       '04:33:05'], dtype=object)

Repetimos el procedimiento para las horas de cierre:

In [50]:
df_hrs_raras = df_incidentes.loc[~df_incidentes["hora_cierre"].str.contains(':', case=False, na=None)]

In [51]:
df_hrs_raras.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida
831387,C5/200218/04419,2020-02-18,14:14:33,Martes,(N) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.635243056,ALVARO OBREGON,accidente-choque sin lesionados,19.35964,-99.209128,EMERGENCIA,LLAMADA DEL 911,ALVARO OBREGON,"19.35964,-99.20912778",2,NaN
831459,C5/200218/02272,2020-02-18,09:49:30,Martes,(D) El incidente reportado se registró en dos ...,2020-02-18,2020,Febrero,0.421828704,MIGUEL HIDALGO,lesionado-atropellado,19.47115,-99.206820,URGENCIAS MEDICAS,LLAMADA APP911,MIGUEL HIDALGO,"19.47114999,-99.20682",2,NaN


In [52]:
len(df_hrs_raras)

1542

In [53]:
hrs_raras = df_hrs_raras['hora_cierre']

In [54]:
hrs_raras

831387     0.635243056
831459     0.421828704
831467     0.979803241
831615     0.634780093
831749     0.585763889
              ...     
1382352    0.047685185
1382353    0.101643519
1382354    0.053009259
1382355    0.048530093
1382356    0.021076389
Name: hora_cierre, Length: 1542, dtype: object

In [55]:
print(datetime.timedelta(days=0.985266204))

23:38:47.000026


In [56]:
print(datetime.timedelta(days=float(list(hrs_raras)[0])))

15:14:45.000038


In [57]:
datetime.timedelta(days=float(list(hrs_raras)[0]))

datetime.timedelta(seconds=54885, microseconds=38)

In [58]:
hrs_raras = hrs_raras.astype('float')
hours = [datetime.timedelta(days=float(num)) for num in hrs_raras]

In [59]:
data_hr_corregida = {'folio':df_hrs_raras['folio'],'hora_cierre':df_hrs_raras['hora_creacion'],'hora_cierre_corregida':hours}

In [60]:
dataframe_hrcorregida = pd.DataFrame(data=data_hr_corregida)

In [61]:
dataframe_hrcorregida = df_hrs_raras.copy()
dataframe_hrcorregida['hora_cierre_corregida'] = hours

In [62]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida,hora_cierre_corregida
831387,C5/200218/04419,2020-02-18,14:14:33,Martes,(N) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.635243056,ALVARO OBREGON,accidente-choque sin lesionados,19.35964,-99.209128,EMERGENCIA,LLAMADA DEL 911,ALVARO OBREGON,"19.35964,-99.20912778",2,NaN,0 days 15:14:45.000038
831459,C5/200218/02272,2020-02-18,09:49:30,Martes,(D) El incidente reportado se registró en dos ...,2020-02-18,2020,Febrero,0.421828704,MIGUEL HIDALGO,lesionado-atropellado,19.47115,-99.206820,URGENCIAS MEDICAS,LLAMADA APP911,MIGUEL HIDALGO,"19.47114999,-99.20682",2,NaN,0 days 10:07:26.000026


In [63]:
# Eliminamos microsegundos
dataframe_hrcorregida['hora_cierre_corregida']= dataframe_hrcorregida['hora_cierre_corregida'].dt.floor('s')

In [64]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida,hora_cierre_corregida
831387,C5/200218/04419,2020-02-18,14:14:33,Martes,(N) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.635243056,ALVARO OBREGON,accidente-choque sin lesionados,19.35964,-99.209128,EMERGENCIA,LLAMADA DEL 911,ALVARO OBREGON,"19.35964,-99.20912778",2,NaN,0 days 15:14:45
831459,C5/200218/02272,2020-02-18,09:49:30,Martes,(D) El incidente reportado se registró en dos ...,2020-02-18,2020,Febrero,0.421828704,MIGUEL HIDALGO,lesionado-atropellado,19.47115,-99.206820,URGENCIAS MEDICAS,LLAMADA APP911,MIGUEL HIDALGO,"19.47114999,-99.20682",2,NaN,0 days 10:07:26


In [65]:
dataframe_hrcorregida.dtypes

folio                               object
fecha_creacion              datetime64[ns]
hora_creacion                       object
dia_semana                          object
codigo_cierre                       object
fecha_cierre                datetime64[ns]
año_cierre                           int64
mes_cierre                          object
hora_cierre                         object
delegacion_inicio                   object
incidente_c4                        object
latitud                            float64
longitud                           float64
clas_con_f_alarma                   object
tipo_entrada                        object
delegacion_cierre                   object
geopoint                            object
mes                                  int64
hora_creacion_corregida             object
hora_cierre_corregida      timedelta64[ns]
dtype: object

In [66]:
#La pasamos a string para eliminar el texto de 0 días
dataframe_hrcorregida['hora_cierre_corregida'] = dataframe_hrcorregida['hora_cierre_corregida'].astype(str)

In [67]:
#Eliminamos el string de 0 dias
dataframe_hrcorregida['hora_cierre_corregida'] = dataframe_hrcorregida['hora_cierre_corregida'].str.replace('0 days ', '')

In [68]:
dataframe_hrcorregida.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida,hora_cierre_corregida
831387,C5/200218/04419,2020-02-18,14:14:33,Martes,(N) La unidad de atención a emergencias fue de...,2020-02-18,2020,Febrero,0.635243056,ALVARO OBREGON,accidente-choque sin lesionados,19.35964,-99.209128,EMERGENCIA,LLAMADA DEL 911,ALVARO OBREGON,"19.35964,-99.20912778",2,NaN,15:14:45
831459,C5/200218/02272,2020-02-18,09:49:30,Martes,(D) El incidente reportado se registró en dos ...,2020-02-18,2020,Febrero,0.421828704,MIGUEL HIDALGO,lesionado-atropellado,19.47115,-99.206820,URGENCIAS MEDICAS,LLAMADA APP911,MIGUEL HIDALGO,"19.47114999,-99.20682",2,NaN,10:07:26


In [69]:
dataframe_hrcorregida['hora_cierre'] = dataframe_hrcorregida['hora_cierre_corregida']

In [70]:
df_incidentes = df_incidentes.append(dataframe_hrcorregida)

In [71]:
df_incidentes.drop_duplicates(keep='last',subset=['folio'],inplace = True)

In [72]:
df_incidentes.head(2)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,hora_creacion_corregida,hora_cierre_corregida
0,GA/160123/05714,2016-01-23,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,2016-01-24,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,"19.422113,-99.084643",1,NaN,NaN
1,AO/160123/05826,2016-01-23,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,2016-01-24,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,"19.35832,-99.297641",1,NaN,NaN


In [73]:
# Verificamos:
df_incidentes['hora_cierre'].unique()

array(['00:21:08', '04:40:37', '11:13:03', ..., '04:21:49', '02:44:31',
       '04:35:21'], dtype=object)

In [74]:
# Eliminamos columnas auxiliares
df_incidentes.drop(columns = 'hora_creacion_corregida',inplace = True)
df_incidentes.drop(columns = 'hora_cierre_corregida',inplace = True)

In [75]:
# Cambiamos tipos de datos
df_incidentes = df_incidentes.astype({"dia_semana":'category',"año_cierre":'category', "codigo_cierre":'category', "año_cierre":'int64', "mes_cierre":'category',"delegacion_inicio":'category', "incidente_c4":'category' , "clas_con_f_alarma":'category', "tipo_entrada":'category',  "delegacion_cierre":'category' , "mes":'int64'}) 

In [76]:
df_incidentes.dtypes

folio                        object
fecha_creacion       datetime64[ns]
hora_creacion                object
dia_semana                 category
codigo_cierre              category
fecha_cierre         datetime64[ns]
año_cierre                    int64
mes_cierre                 category
hora_cierre                  object
delegacion_inicio          category
incidente_c4               category
latitud                     float64
longitud                    float64
clas_con_f_alarma          category
tipo_entrada               category
delegacion_cierre          category
geopoint                     object
mes                           int64
dtype: object

In [77]:
df_incidentes['año_cierre'] = df_incidentes['año_cierre'].astype('category')
df_incidentes['mes'] = df_incidentes['mes'].astype('category')

### Perfilamiento por variables

In [78]:
profiling_numericas,profiling_categoricas,profiling_texto = eda.genera_profiling_por_variable(df_incidentes)

In [79]:
profiling_numericas

,metrica,latitud,longitud
0,tipo,float64,float64
1,numero de observaciones,1.3827e+06,1.3827e+06
2,media,19.38,-99.14
3,desviacion estándar,0.27,2.4
4,cuartil 25%,19.34,-99.18
5,cuartil 50%,19.38,-99.14
6,cuartil 75%,19.43,-99.1
7,minimo,19.09,-991.76
8,maximo,195.3,-98.95
9,numero de observaciones unicas,82501,78984


In [80]:
profiling_categoricas

,metrica,dia_semana,codigo_cierre,año_cierre,mes_cierre,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,delegacion_cierre,mes
0,tipo,category,category,category,category,category,category,category,category,category,category
1,numero de categorias,7,5,7,12,16,26,4,9,16,12
2,numero de observaciones,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138
3,observaciones nulas,0,0,0,0,158,0,0,0,140,0
4,% observaciones nulas,0,0,0,0,0.000114233,0,0,0,0.000101219,0
5,valores unicos,"[Sábado, Domingo, Lunes, Martes, Miércoles, Ju...",[(A) La unidad de atención a emergencias fue d...,"[2016, 2017, 2018, 2019, 2015, 2014, 2020]","[Enero, Febrero, Marzo, Septiembre, Octubre, M...","[VENUSTIANO CARRANZA, CUAJIMALPA, TLALPAN, MAG...","[accidente-choque sin lesionados, accidente-ch...","[EMERGENCIA, URGENCIAS MEDICAS, FALSA ALARMA, ...","[BOTÓN DE AUXILIO, LLAMADA DEL 066, RADIO, CÁM...","[VENUSTIANO CARRANZA, CUAJIMALPA, TLALPAN, MAG...","[1, 2, 3, 9, 10, 5, 6, 4, 8, 12, 7, 11]"
6,moda1/veces/porcentaje,"[Viernes, 229,681, 16.61%]",[(A) La unidad de atención a emergencias fue d...,"[2018, 246,577, 17.83%]","[Octubre, 133,641, 9.66%]","[IZTAPALAPA, 223,687, 16.17%]","[accidente-choque sin lesionados, 760,506, 54....","[EMERGENCIA, 799,037, 57.77%]","[LLAMADA DEL 911, 739,339, 53.45%]","[IZTAPALAPA, 225,202, 16.28%]","[10, 133,641, 9.66%]"
7,moda2/veces/porcentaje,"[Sábado, 214,134, 15.48%]",[(D) El incidente reportado se registró en dos...,"[2019, 229,467, 16.59%]","[Agosto, 121,123, 8.76%]","[GUSTAVO A. MADERO, 144,620, 10.46%]","[accidente-choque con lesionados, 315,031, 22....","[URGENCIAS MEDICAS, 572,619, 41.4%]","[LLAMADA DEL 066, 463,432, 33.51%]","[GUSTAVO A. MADERO, 147,582, 10.67%]","[8, 121,123, 8.76%]"
8,moda3/veces/porcentaje,"[Jueves, 198,147, 14.33%]",[(N) La unidad de atención a emergencias fue d...,"[2017, 226,535, 16.38%]","[Septiembre, 120,581, 8.72%]","[CUAUHTEMOC, 141,391, 10.22%]","[lesionado-atropellado, 193,553, 13.99%]","[FALSA ALARMA, 10,630, 0.77%]","[BOTÓN DE AUXILIO, 78,630, 5.68%]","[CUAUHTEMOC, 141,781, 10.25%]","[9, 120,581, 8.72%]"


Regresar mes a numérica

Creamos la variable de timestamp

In [81]:
df_incidentes['hora_creacion'] = df_incidentes['hora_creacion'].astype('str')

In [82]:
df_incidentes['fecha_creacion'] = df_incidentes['fecha_creacion'].astype('str')

In [83]:
df_incidentes['timestamp_creacion'] = df_incidentes['fecha_creacion']+' '+df_incidentes['hora_creacion']

In [84]:
# convert the 'Date' column to datetime format
df_incidentes['timestamp_creacion']= pd.to_datetime(df_incidentes['timestamp_creacion'])

In [85]:
df_incidentes['hora_cierre'] = df_incidentes['hora_cierre'].astype('str')

In [86]:
df_incidentes['fecha_cierre'] = df_incidentes['fecha_cierre'].astype('str')

In [87]:
df_incidentes['timestamp_cierre'] = df_incidentes['fecha_cierre']+' '+df_incidentes['hora_cierre']

In [88]:
# convert the 'Date' column to datetime format
df_incidentes['timestamp_cierre']= pd.to_datetime(df_incidentes['timestamp_cierre'])

In [89]:
diferencia_de_tiempo = df_incidentes['timestamp_cierre'] - df_incidentes['timestamp_creacion']

In [90]:
diferencia_de_tiempo.min()

Timedelta('-1 days +23:59:00')

In [91]:
diferencia_de_tiempo.max()

Timedelta('397 days 03:59:16')

In [92]:
df_incidentes['dif_solucion'] = diferencia_de_tiempo

In [93]:
df_incidentes.sample()

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,...,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,timestamp_creacion,timestamp_cierre,dif_solucion
315800,C5/160428/02284,2016-04-28,12:13:08,Jueves,(N) La unidad de atención a emergencias fue de...,2016-04-28,2016,Abril,14:07:31,ALVARO OBREGON,...,19.35606,-99.2392,EMERGENCIA,LLAMADA DEL 066,ALVARO OBREGON,"19.35605997,-99.23919984",4,2016-04-28 12:13:08,2016-04-28 14:07:31,0 days 01:54:23


In [94]:
df_incidentes.iloc[df_incidentes['dif_solucion'].idxmin()]

folio                                                   C5/190820/07924
fecha_creacion                                               2019-08-20
hora_creacion                                                  23:37:03
dia_semana                                                       Martes
codigo_cierre         (D) El incidente reportado se registró en dos ...
fecha_cierre                                                 2019-08-20
año_cierre                                                         2019
mes_cierre                                                       Agosto
hora_cierre                                                    23:45:10
delegacion_inicio                                            XOCHIMILCO
incidente_c4                            accidente-choque con lesionados
latitud                                                         19.2525
longitud                                                       -99.1014
clas_con_f_alarma                                     URGENCIAS 

In [95]:
df_incidentes['timestamp_creacion'].max()

Timestamp('2020-10-31 23:57:20')

In [96]:
df_incidentes['timestamp_creacion'].min()

Timestamp('2013-12-31 13:01:33')

In [97]:
#df_incidentes['timestamp_creacion']

In [98]:
df_incidentes['hora_creacion'].unique()

array(['22:35:04', '22:50:49', '09:40:11', ..., '03:08:57', '02:05:12',
       '04:33:05'], dtype=object)

In [99]:
df_incidentes.loc[df_incidentes["latitud"] > 20]

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,...,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,timestamp_creacion,timestamp_cierre,dif_solucion
284145,C5/181231/06094,2018-12-31,17:03:39,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,17:21:08,BENITO JUAREZ,...,193.815,NaN,URGENCIAS MEDICAS,LLAMADA DEL 911,BENITO JUAREZ,NaN,12,2018-12-31 17:03:39,2018-12-31 17:21:08,0 days 00:17:29
436981,C5/181231/06971,2018-12-31,18:46:55,Lunes,(N) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,19:00:13,IZTAPALAPA,...,193.446,NaN,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,NaN,12,2018-12-31 18:46:55,2018-12-31 19:00:13,0 days 00:13:18
714661,GA/181231/04533,2018-12-31,14:15:47,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,17:16:38,GUSTAVO A. MADERO,...,195.303,NaN,EMERGENCIA,BOTÓN DE AUXILIO,GUSTAVO A. MADERO,NaN,12,2018-12-31 14:15:47,2018-12-31 17:16:38,0 days 03:00:51


In [100]:
df_incidentes.loc[df_incidentes["longitud"] < -100]

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,...,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes,timestamp_creacion,timestamp_cierre,dif_solucion
284110,C5/181231/01210,2018-12-31,06:02:10,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,06:05:35,IZTAPALAPA,...,NaN,-990.369,URGENCIAS MEDICAS,LLAMADA DEL 911,IZTAPALAPA,NaN,12,2018-12-31 06:02:10,2018-12-31 06:05:35,0 days 00:03:25
284125,C5/181231/05350,2018-12-31,15:42:11,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,15:50:06,CUAUHTEMOC,...,NaN,-991.692,URGENCIAS MEDICAS,LLAMADA DEL 911,CUAUHTEMOC,NaN,12,2018-12-31 15:42:11,2018-12-31 15:50:06,0 days 00:07:55
436964,C5/181231/06210,2018-12-31,17:18:40,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,19:09:52,IZTACALCO,...,NaN,-990.765,URGENCIAS MEDICAS,RADIO,IZTACALCO,NaN,12,2018-12-31 17:18:40,2018-12-31 19:09:52,0 days 01:51:12
436990,C5/181231/01068,2018-12-31,04:51:42,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,05:18:29,AZCAPOTZALCO,...,NaN,-991.764,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,NaN,12,2018-12-31 04:51:42,2018-12-31 05:18:29,0 days 00:26:47
437000,C5/181231/05354,2018-12-31,15:42:14,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,15:49:03,CUAUHTEMOC,...,NaN,-991.692,URGENCIAS MEDICAS,LLAMADA DEL 911,CUAUHTEMOC,NaN,12,2018-12-31 15:42:14,2018-12-31 15:49:03,0 days 00:06:49
437042,C5/181231/05353,2018-12-31,15:41:59,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,15:49:27,CUAUHTEMOC,...,NaN,-991.692,URGENCIAS MEDICAS,LLAMADA DEL 911,CUAUHTEMOC,NaN,12,2018-12-31 15:41:59,2018-12-31 15:49:27,0 days 00:07:28
437075,C5/181231/00961,2018-12-31,04:08:37,Lunes,(D) El incidente reportado se registró en dos ...,2018-12-31,2018,Diciembre,04:18:48,AZCAPOTZALCO,...,NaN,-991.764,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,NaN,12,2018-12-31 04:08:37,2018-12-31 04:18:48,0 days 00:10:11
714449,C5/181231/05744,2018-12-31,16:22:20,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,18:10:39,IZTAPALAPA,...,NaN,-990.495,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,NaN,12,2018-12-31 16:22:20,2018-12-31 18:10:39,0 days 01:48:19
714483,C5/181231/00956,2018-12-31,04:06:15,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,06:22:55,AZCAPOTZALCO,...,NaN,-991.764,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,NaN,12,2018-12-31 04:06:15,2018-12-31 06:22:55,0 days 02:16:40
714513,C5/181231/04311,2018-12-31,13:34:03,Lunes,(A) La unidad de atención a emergencias fue de...,2018-12-31,2018,Diciembre,16:01:07,TLAHUAC,...,NaN,-989.595,EMERGENCIA,LLAMADA DEL 911,TLAHUAC,NaN,12,2018-12-31 13:34:03,2018-12-31 16:01:07,0 days 02:27:04


In [ ]:
eda.CreaTablaConteoPorcentaje(df_delitos,"año_cierre", False)